In [1]:
# reloads modules when running again
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy

from model_functions import *

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from darts import TimeSeries
from datetime import datetime
from sklearn.metrics.pairwise import pairwise_distances

# TODO: delete
from darts.models.forecasting.varima import VARIMA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

## Data loading

In [3]:
data, metadata = load_data()

### Constants

In [4]:
test_size = 10

## Data preprocessing

### Split to train-test

In [5]:
# sort data by collection date
data = data.sort_values(by="collection_date")

# split to train & test by test_size last samples
test_df = data.groupby('baboon_id').tail(test_size)
train_df = data.drop(test_df.index)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

# TODO: remove
test_df = test_df[test_df["baboon_id"].isin(["Baboon_101", "Baboon_107", "Baboon_201"])]
train_df = train_df[train_df["baboon_id"].isin(["Baboon_101", "Baboon_107", "Baboon_201"])]

In [6]:
meta_features_union = meta_features + ["month_sin", "month_cos"]
x_test, y_test = test_df[meta_features_union], test_df.drop(columns=meta_features_union)
x_test["interpolated"] = 0

### Interpolation

In [7]:
# aggregate to one sample per week
train_df = aggregate_samples(train_df.copy())

In [8]:
# TODO: delete i in knn_interpolation func
interpolated_df = knn_interpolation(train_df)
interpolated_df.to_csv("interpolated_df.csv")

1
2
3


In [9]:
interpolated_df["collection_date"] = pd.to_datetime(interpolated_df["collection_date"])

### Format dataset

### Add random noise for non-singularity

In [10]:
taxa_columns = [col for col in interpolated_df.columns if col not in ["sample", "baboon_id", "collection_date", "interpolated"]]
# min_value = interpolated_df[taxa_columns][interpolated_df[taxa_columns] > 0].min().min()
# noise = np.random.uniform(0.25 * min_value, min_value, size=interpolated_df[taxa_columns].shape)

In [11]:
# noise_df = interpolated_df.copy()
# noise_df[taxa_columns] = interpolated_df[taxa_columns] + noise
# #noise_df[taxa_columns] = noise_df[taxa_columns] * 1000
# noise_df[taxa_columns] = noise_df[taxa_columns].div(noise_df[taxa_columns].sum(axis=1), axis=0)

## Train

In [12]:
pred_df = predict(train_df, x_test)

## Performance analysis

In [13]:
d_matrix = pairwise_distances(train_df[y_test.columns], y_test, metric="braycurtis")
pd.DataFrame(d_matrix).to_csv("braycurtis_performance.csv")
print(d_matrix.mean())

0.49276818405774553
